In [1]:
import torch
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
import collections as ct
import time
import utils

Using TensorFlow backend.


In [2]:
sentences = pd.read_csv("data/sentences.csv")
sentences.head()

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,What is the step by step guide to invest in sh...,2054,2003,1996,3357,2011,3357,5009,2000,15697,...,0,0,0,0,0,0,0,0,0,0
1,What is the step by step guide to invest in sh...,2054,2003,1996,3357,2011,3357,5009,2000,15697,...,0,0,0,0,0,0,0,0,0,0
2,What is the story of Kohinoor (Koh-i-Noor) Dia...,2054,2003,1996,2466,1997,12849,10606,16506,1006,...,0,0,0,0,0,0,0,0,0,0
3,What would happen if the Indian government sto...,2054,2052,4148,2065,1996,2796,2231,10312,1996,...,0,0,0,0,0,0,0,0,0,0
4,How can I increase the speed of my internet co...,2129,2064,1045,3623,1996,3177,1997,2026,4274,...,0,0,0,0,0,0,0,0,0,0


In [3]:
embeddings = sentences.iloc[:,1:]
embeddings = embeddings.values.tolist()

In [4]:
model_class = BertForSequenceClassification
tokenizer_class = BertTokenizer
pretrained_model = 'bert-base-uncased'
tokenizer = tokenizer_class.from_pretrained(pretrained_model)

In [5]:
print(tokenizer.decode([2053]))
print(tokenizer.decode([2054]))
print(tokenizer.decode([4301]))
print(tokenizer.decode([4302]))

no
what
thoughts
harry


In [9]:
def embed (sent):
    # returns the embedding, standardized to length 34, as the sentences in 'embeddings' are this long
    emb = tokenizer.encode(sent, add_special_tokens=False)
    
    if len(emb) > 34:
        emb = emb[:34]
    else:
        emb += [0] * (34 - len(emb))

    return emb

sent = "Is Harry?"
print(embed(sent))

[2003, 4302, 1029, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
def most_similar_v1(sent):
    '''
    It finds the most similar sentence in the dataset
    - input: sentence consisting in a string
    - output: index of the most similar sentence in the dataset
    '''
    emb = embed(sent)
    d = cdist(embeddings, [emb])

    present = False
    i = D = np.inf
    for j in range(len(d)):
        if d[j] == 0:
            present = True
        elif d[j] < D:
            D = d[j]
            i = j
        
    print("Most similar sentence to: ", sent)
    if present:
        print("The sentence is present in the list")
    print("index: ", i)
    print("sentence: ", sentences.iloc[i,0])
    print("distance: ", float(D),"\n")
    
    return i


def similarities(emb):
    # Returns the scaled similarities between a sample embedding and the ones in embedding
    sample = ct.Counter(emb)
    del sample[0]
    sim = [None] * len(embeddings)
    for i in range(len(embeddings)):
        ref = ct.Counter(embeddings[i])
        del ref[0]
        n = max(sum(sample.values()), sum(ref.values()))
        s = 0
        for el in sample:
            s += min(sample[el], ref[el])
        sim[i] = s/n*100
    return sim


def most_similar(sent):
    '''
    It finds the most similar sentence in the dataset
    - input: sentence consisting in a string
    - output: index of the most similar sentence in the dataset
    '''
    emb = embed(sent)
    sim = similarities(emb)

    present = False
    i = s = -np.inf
    for j in range(len(sim)):
        if sim[j] == 100:
            present = True
        elif sim[j] > s:
            s = sim[j]
            i = j
        
    print("Most similar sentence to: ", sent)
    if present:
        print("The sentence is present in the list!")
    print("index: ", i)
    print("sentence: ", sentences.iloc[i,0])
    print("similarity: ", "{:.2f}".format(s), "%\n", sep="")
    
    return i

In [8]:
sent = "Who was more voted in the presidential elections, Hillary Clinton or Donald Trump?"
time1 = time.time()
_ = most_similar_v1(sent)
time2 = time.time()
_ = most_similar(sent)
time3 = time.time()

print("First version lasted: ", utils.format_time(time2-time1))
print("Second version lasted: ", utils.format_time(time3-time2), "\n \n")


sent = "Is Harry Potter in love with Hermione?"
_ = most_similar_v1(sent)
_ = most_similar(sent)
print("")

sent = "Which is the best rock band in history?"
_ = most_similar_v1(sent)
i = most_similar(sent)
_ = most_similar(sentences.iloc[i,0])

Most similar sentence to:  Who was more voted in the presidential elections, Hillary Clinton or Donald Trump?
index:  32422
sentence:  Who will be the next president of USA: Hillary Clinton or Donald Trump?
distance:  4519.969247682997 

Most similar sentence to:  Who was more voted in the presidential elections, Hillary Clinton or Donald Trump?
index:  80453
sentence:  Who will win the US presidential elections 2016: Hillary Clinton or Donald Trump?
similarity: 66.67%

First version lasted:  0:00:22
Second version lasted:  0:00:47 
 

Most similar sentence to:  Is Harry Potter in love with Hermione?
index:  260736
sentence:  What does IMO mean in a text message?
distance:  2550.2176377713336 

Most similar sentence to:  Is Harry Potter in love with Hermione?
index:  156559
sentence:  Why don't Hermione fall in love with Harry?
similarity: 61.54%


Most similar sentence to:  Which is the best rock band in history?
index:  225975
sentence:  What are the best video games to play?
distanc